In [7]:
# https://arxiv.org/pdf/1912.08777.pdf
!pip install transformers sentencepiece
!pip install sacremoses
from transformers import *

model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


loading configuration file https://huggingface.co/tuner007/pegasus_paraphrase/resolve/main/config.json from cache at /home/username/.cache/huggingface/transformers/d6a784b31cbe212ac3dabbc78bc4e454cf9d8a1b11ff2ef1ba1c7497f2bbfb33.7a3e093739f407bcc025e64dfc5244f91bc573ed4285cba53de90f960cbce58e
Model config PegasusConfig {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token_to_be_generated": false,
 

In [21]:
import csv

runs = 2

pdf_files = ['Cancellation', 'SecureTripBudget','SecureTripClassic', 'SecureTripPremium', 'SecureTripPremiumPlus', 'TravelProtectionClassic', 'TravelProtectionPremium', 'TravelProtectionPremiumPlus' ]
for j in range(len(pdf_files)):
    with open('./adobe_csv/'+pdf_files[j]+'.csv', newline='') as csvfile:
        reader = list(csv.reader(csvfile, quoting=csv.QUOTE_ALL, skipinitialspace=True))        
        
    augmentation = []
    for i, row in enumerate(reader):
        if i ==0 :
            header = row
            continue

        questions = get_paraphrased_sentences(model, tokenizer, row[1], num_beams=runs, num_return_sequences=runs)
        answers = get_paraphrased_sentences(model, tokenizer, row[2], num_beams=runs, num_return_sequences=runs)
        augmentation.append(row)
        for i in range(runs):
            augmentation.append([row[0], questions[i], answers[i], row[3]])
    
    with open('./adobe_csv_augmentation/'+pdf_files[j]+'_augmentation.csv', mode= 'w',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(augmentation)